# Web Scraping

This notebook will have problems that touch on the material covered in `Lectures/Data Collection/Web Scraping with BeautifulSoup`.

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from time import sleep

##### 1. Asthma

Scrape the percent of each age group that has asthma according to these CDC statistics, <a href="https://www.cdc.gov/asthma/most_recent_national_asthma_data.htm">https://www.cdc.gov/asthma/most_recent_national_asthma_data.htm</a>. Turn the data into a `DataFrame`, which group has the highest rate, the lowest rate? (Answer this question about rates using `pandas`)

##### Sample Solution

In [2]:
url = "https://www.cdc.gov/asthma/most_recent_national_asthma_data.htm"
html = requests.get(url)
soup = BeautifulSoup(html.content, 'html.parser')

In [3]:
table = soup.find('table').tbody

In [4]:
ages = [td.text for td in table.find_all('td', {'headers':"characteristic all_age"})]
percs_point = [float(td.text.split(" ")[0]) for td in table.find_all('td', {'headers':"percent all_age"})]
percs_se = [float(td.text.split(" ")[1].replace("(","").replace(")","")) for td in table.find_all('td', {'headers':"percent all_age"})]

In [5]:
asthma = pd.DataFrame({'age_group':ages,
                         'percs_point':percs_point,
                         'percs_se':percs_se})

asthma

,age_group,percs_point,percs_se
0,0–4,1.9,0.34
1,5–11,7.5,0.58
2,5–14,7.7,0.47
3,5-17 (School Age),8.1,0.42
4,12-14 (Young Teens),8.0,0.84
5,12-17,8.7,0.63
6,15-17 (Teenagers),9.5,0.88
7,15–19,9.0,0.76
8,11-21 (Adolescents),8.4,0.52
9,20–24,9.5,0.96


In [6]:
asthma.loc[asthma.percs_point==asthma.percs_point.max()]

,age_group,percs_point,percs_se
6,15-17 (Teenagers),9.5,0.88
9,20–24,9.5,0.96


##### 2. Let's go Cavs!

Write a script to get the scores from all of the Cleveland Cavaliers games from this site, <a href="https://www.basketball-reference.com/teams/CLE/2022_games.html">https://www.basketball-reference.com/teams/CLE/2022_games.html</a>, these are scored in the `Tm` and `Opp` columns.

In [7]:
url = "https://www.basketball-reference.com/teams/CLE/2022_games.html"
html = requests.get(url)
soup = BeautifulSoup(html.content, 'html.parser')

In [8]:
table = soup.find('table', {'id':'games'}).tbody

In [9]:
pts_scored = [td.text for td in table.find_all('td', {'data-stat':"pts"})]
opp_pts = [td.text for td in table.find_all('td', {'data-stat':"opp_pts"})]

pd.DataFrame({'point_scored':pts_scored,
                 'opp_scored':opp_pts})

,point_scored,opp_scored
0,121,132
1,112,123
2,101,95
3,99,87
4,92,79
...,...,...
79,115,120
80,107,118
81,133,115
82,108,115


##### 3. More Scores

Repeat what you did in problem 2, but this time for all seasons from the 2000-01 to the 2021-22 seasons.

Also record the opponent team name, the season and the scores in a single `DataFrame`. In which season did the Cleveland Cavaliers score the highest average points per game?

In [10]:
from time import sleep
import numpy as np

In [11]:
years = range(2001, 2023)

base_url = "https://www.basketball-reference.com/teams/CLE/"
end_of_url = "_games.html"

opponents = []
seasons = []
pts_scored = []
opp_pts = []

for year in years:
    print("Working on", year)
    html = requests.get(base_url + str(year) + end_of_url)
    soup = BeautifulSoup(html.content, 'html.parser')
    
    table = soup.find('table', {'id':'games'}).tbody
    
    season_opp_names = [td.text for td in table.find_all('td', {'data-stat':"opp_name"})]
    season_pts_scored = [int(td.text) for td in table.find_all('td', {'data-stat':"pts"})]
    season_opp_pts = [int(td.text) for td in table.find_all('td', {'data-stat':"opp_pts"})]
    season_seasons = np.repeat(str(year-1) + "-" + str(year)[-2:], len(season_opp_names))

    opponents.extend(season_opp_names)
    seasons.extend(season_seasons)
    pts_scored.extend(season_pts_scored)
    opp_pts.extend(season_opp_pts)
    
    sleep(3)

Working on 2001
Working on 2002
Working on 2003
Working on 2004
Working on 2005
Working on 2006
Working on 2007
Working on 2008
Working on 2009
Working on 2010
Working on 2011
Working on 2012
Working on 2013
Working on 2014
Working on 2015
Working on 2016
Working on 2017
Working on 2018
Working on 2019
Working on 2020
Working on 2021
Working on 2022


In [12]:
cavs_scores = pd.DataFrame({'season':seasons,
                                'opponent':opponents,
                                'pts_scored':pts_scored,
                                'opp_pts':opp_pts})

In [13]:
cavs_scores.groupby('season').pts_scored.mean().sort_values(ascending=False)

season
2017-18    110.865854
2016-17    110.341463
2021-22    107.714286
2019-20    106.892308
2018-19    104.475610
2015-16    104.329268
2020-21    103.833333
2014-15    103.134146
2009-10    102.109756
2008-09    100.280488
2013-14     98.219512
2005-06     97.585366
2006-07     96.756098
2004-05     96.512195
2012-13     96.500000
2007-08     96.378049
2010-11     95.451220
2001-02     95.268293
2011-12     93.030303
2003-04     92.914634
2000-01     92.207317
2002-03     91.402439
Name: pts_scored, dtype: float64

##### 4. Scraping Scientific Articles

You are working on a research project that involves analyzing the posting of different journal article links. You have been tasked with scraping the title, authors and dois given a list of urls linking to different journal articles. There are three unique domains included in the data loaded for you below.

In [14]:
articles = pd.read_csv("journal_article_urls.csv")

In [15]:
articles.head()

,domain,url
0,www.science.org,https://www.science.org/doi/10.1126/sciimmunol...
1,www.science.org,https://www.science.org/doi/10.1126/scisignal....
2,www.science.org,https://www.science.org/doi/10.1126/sciimmunol...
3,www.science.org,https://www.science.org/doi/10.1126/scitranslm...
4,www.science.org,https://www.science.org/doi/10.1126/science.ab...


In [16]:
articles.domain.value_counts()

www.science.org      10
www.nature.com       10
www.thelancet.com     8
Name: domain, dtype: int64

##### Nature

First write a function that can scrape the urls from `www.nature.com`

##### Sample Solution

In [25]:
def nature(url):
    html = requests.get(url)
    soup = BeautifulSoup(html.content)
    
    ## Title
    if soup.title:
        if soup.title.text:
            title = soup.title.text.split("|")[0].strip()
        else:
            title = "NA"
    else:
        title = "NA"

    ## Authors
    if soup.find_all('a', {'data-test':"author-name"}):
        authors = ", ".join([a.text for a in soup.find_all('a', {'data-test':"author-name"})])
    else:
        authors = "NA"
        
        
    return title, authors

In [26]:
for url in articles.loc[articles.domain=='www.nature.com'].url.values:
    print(url)
    title, authors = nature(url)
    print(title)
    print(authors)
    print()
    sleep(3)

https://www.nature.com/articles/s41586-022-04629-w
Projected environmental benefits of replacing beef with microbial protein
Florian Humpenöder, Benjamin Leon Bodirsky, Isabelle Weindl, Hermann Lotze-Campen, Tomas Linder, Alexander Popp

https://www.nature.com/articles/s41586-022-04617-0
Protected areas have a mixed impact on waterbirds, but management helps
Hannah S. Wauchope, Julia P. G. Jones, Jonas Geldmann, Benno I. Simmons, Tatsuya Amano, Daniel E. Blanco, Richard A. Fuller, Alison Johnston, Tom Langendoen, Taej Mundkur, Szabolcs Nagy, William J. Sutherland

https://www.nature.com/articles/s41586-022-04666-5
Nonlinear mechanics of human mitotic chromosomes
Anna E. C. Meijering, Kata Sarlós, Christian F. Nielsen, Hannes Witt, Janni Harju, Emma Kerklingh, Guus H. Haasnoot, Anna H. Bizard, Iddo Heller, Chase P. Broedersz, Ying Liu, Erwin J. G. Peterman, Ian D. Hickson, Gijs J. L. Wuite

https://www.nature.com/articles/s41586-022-04539-x
Single electrons on solid neon as a solid-stat

##### Science

Try to request the html code for the first url from the `www.science.org` domain.

What is the status response code?

##### Sample Solution

In [22]:
url = articles.loc[articles.domain=='www.science.org'].url.values[0]

requests.get(url)

<Response [403]>

When I was writing this notebook, I received a 503 response. This code implies an issue on the website's side and not an issue with your code. While there are different reasons why I got this response, I believe it is because `www.science.org`'s servers have been set up to prevent scraping like we are trying to accomplish.

Luckily, we have another way to get the data we want for these urls, which we will touch on in the `Python and APIs` `Practice Problems` notebook.

--------------------------

This notebook was written for the Erd&#337;s Institute C&#337;de Data Science Boot Camp by Matthew Osborne, Ph. D., 2023.

Any potential redistributors must seek and receive permission from Matthew Tyler Osborne, Ph.D. prior to redistribution. Redistribution of the material contained in this repository is conditional on acknowledgement of Matthew Tyler Osborne, Ph.D.'s original authorship and sponsorship of the Erdős Institute as subject to the license (see License.md)